# TSA Chapter 3: ARIMA Model Diagnostics

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch3/TSA_ch3_case_diagnostics/TSA_ch3_case_diagnostics.ipynb)

This notebook demonstrates:
- Residual analysis for ARIMA model fitted on US Real GDP data.

In [ ]:
!pip install numpy pandas matplotlib statsmodels pandas-datareader scipy -q

In [ ]:
import numpy as npimport pandas as pdimport matplotlib.pyplot as pltimport pandas_datareader as pdrfrom statsmodels.tsa.arima.model import ARIMAfrom statsmodels.graphics.tsaplots import plot_acffrom scipy import stats

In [ ]:
# Chart style settings - Nature journal qualityplt.rcParams['figure.facecolor'] = 'none'plt.rcParams['axes.facecolor'] = 'none'plt.rcParams['savefig.facecolor'] = 'none'plt.rcParams['axes.grid'] = Falseplt.rcParams['font.family'] = 'sans-serif'plt.rcParams['font.sans-serif'] = ['Helvetica', 'Arial', 'DejaVu Sans']plt.rcParams['font.size'] = 8plt.rcParams['axes.labelsize'] = 9plt.rcParams['axes.titlesize'] = 10plt.rcParams['xtick.labelsize'] = 8plt.rcParams['ytick.labelsize'] = 8plt.rcParams['legend.fontsize'] = 8plt.rcParams['legend.facecolor'] = 'none'plt.rcParams['legend.framealpha'] = 0plt.rcParams['axes.spines.top'] = Falseplt.rcParams['axes.spines.right'] = Falseplt.rcParams['axes.linewidth'] = 0.5plt.rcParams['lines.linewidth'] = 0.75BLUE, RED = '#1A3A6E', '#DC3545'# Get data# Fit best model# Residuals over time# Histogram# ACF of residuals# Q-Q plot with 45-degree reference lineplt.suptitle('Diagnostic Plots: ARIMA(1,1,1)', fontweight='bold', fontsize=14, y=1.02)plt.tight_layout()plt.savefig('ch3_case_diagnostics.pdf', dpi=300, bbox_inches='tight')plt.show()

In [ ]:
gdp = pdr.get_data_fred('GDPC1', start='1960-01-01', end='2024-09-30')
gdp_data = gdp['GDPC1'].dropna()
log_gdp = np.log(gdp_data)
model = ARIMA(log_gdp, order=(1, 1, 1))
fit = model.fit()
residuals = fit.resid
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes[0, 0].plot(residuals.index, residuals.values, color=BLUE, linewidth=0.8)
axes[0, 0].axhline(y=0, color='red', linestyle='--', linewidth=1)
axes[0, 0].set_title('Residuals Over Time', fontweight='bold')
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('Residual')
axes[0, 1].hist(residuals, bins=30, color=BLUE, edgecolor='white', density=True)
axes[0, 1].set_title('Residual Distribution', fontweight='bold')
axes[0, 1].set_xlabel('Residual')
axes[0, 1].set_ylabel('Density')
plot_acf(residuals, lags=20, ax=axes[1, 0], color=BLUE)
axes[1, 0].set_title('ACF of Residuals', fontweight='bold')
res_sorted = np.sort(residuals)
norm_quantiles = stats.norm.ppf(np.linspace(0.01, 0.99, len(res_sorted)))
axes[1, 1].scatter(norm_quantiles, (res_sorted - res_sorted.mean()) / res_sorted.std(),
                   color=BLUE, s=20, alpha=0.7)
lims = [min(norm_quantiles.min(), -3), max(norm_quantiles.max(), 3)]
axes[1, 1].plot(lims, lims, color=RED, linewidth=1.5, linestyle='--')
axes[1, 1].set_xlim(lims)
axes[1, 1].set_ylim(lims)
axes[1, 1].set_xlabel('Theoretical Quantiles')
axes[1, 1].set_ylabel('Sample Quantiles')
axes[1, 1].set_title('Q-Q Plot', fontweight='bold')
axes[1, 1].set_aspect('equal')